In [1]:
from music21 import converter, chord, note, instrument, duration, stream
import glob
import numpy as np

In [2]:
seq_len=32

In [3]:
cello_path_list=glob.glob("/content/drive/MyDrive/MIDI/suites_for_solo_cello/**/*.mid", recursive=True)
notes_list=[]
durations_list=[]

for cello_path in cello_path_list:
    notes = []
    durations = []
    original_score = converter.parse(cello_path).chordify()
    for element in original_score.flat:
        
        if isinstance(element, chord.Chord):
            notes.append('.'.join(n.nameWithOctave for n in element.pitches))
            durations.append(element.duration.quarterLength)

        if isinstance(element, note.Note):
            if element.isRest:
                notes.append(str(element.name))
                durations.append(element.duration.quarterLength)
            else:
                notes.append(str(element.nameWithOctave))
                durations.append(element.duration.quarterLength)
    notes=["START"]*(seq_len-1)+notes+["END"]
    durations=[0]*(seq_len-1)+durations+[0]
    notes_list.append(notes)
    durations_list.append(durations)

In [4]:
#　辞書を作る
unique_notes=sorted(set(sum(notes_list, [])))
unique_durations=sorted(set(sum(durations_list, [])))

note2num_dic={}
num2note_dic={}
for i, note_i in enumerate(unique_notes):
    note2num_dic[note_i]=i
    num2note_dic[i]=note_i

duration2num_dic={}
num2duration_dic={}
for i, duration_i in enumerate(unique_durations):
    duration2num_dic[duration_i]=i
    num2duration_dic[i]=duration_i

In [5]:
notes_list=[[note2num_dic[note_i] for note_i in notes] for notes in notes_list]
durations_list=[[duration2num_dic[duration_i] for duration_i in durations] for durations in durations_list]

In [6]:
from keras.utils import np_utils

In [7]:
notes_network_input=[]
notes_network_output=[]
durations_network_input=[]
durations_network_output=[]

for notes in notes_list:
    for i in range(len(notes)-seq_len):
         notes_network_input.append(notes[i:i+seq_len])
         notes_network_output.append(notes[i+seq_len])

for duration_i in durations_list:
    for i in range(len(duration_i)-seq_len):
         durations_network_input.append(duration_i[i:i+seq_len])
         durations_network_output.append(duration_i[i+seq_len])

notes_network_input=np.array(notes_network_input)
durations_network_input=np.array(durations_network_input)
notes_network_output=np_utils.to_categorical(notes_network_output)
durations_network_output=np_utils.to_categorical(durations_network_output)

In [8]:
from tensorflow.keras.layers import LSTM, Input, Dropout, Dense, Activation, Embedding, Concatenate, Reshape
from tensorflow.keras.layers import Flatten, RepeatVector, Permute, TimeDistributed
from tensorflow.keras.layers import Multiply, Lambda, Softmax
import tensorflow.keras.backend as K 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop, Adam

In [9]:
embed_size=100
rnn_units=256
use_attention=True
n_notes=notes_network_output.shape[1]
n_durations=durations_network_output.shape[1]

In [10]:
note_input=Input(shape=(None,))
duration_input=Input(shape=(None,))

x1=Embedding(n_notes, embed_size)(note_input)
x2=Embedding(n_durations, embed_size)(duration_input)

x=Concatenate()([x1, x2])

x=LSTM(rnn_units, return_sequences=True)(x)
x=Dropout(0.2)(x)

if use_attention:
    y=LSTM(rnn_units, return_sequences=True)(x)
    x=Dropout(0.2)(y)
    x=Dense(1, activation="tanh")(x)
    x=Reshape([-1])(x)
    alpha=Activation("softmax")(x)
    x=RepeatVector(rnn_units)(alpha)
    x=Permute((2, 1))(x)

    x=Multiply()([x, y])
    x=Lambda(lambda input: K.sum(input, axis=1), output_shape=(rnn_units))(x)
else:
    x=LSTM(rnn_units)(x)
    x=Dropout(0.2)(x)
note_output=Dense(n_notes, activation="softmax")(x)
duration_output=Dense(n_durations, activation="softmax")(x)

In [11]:
model=Model([note_input, duration_input], [note_output, duration_output])
att_model=Model([note_input, duration_input], alpha)

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 100)    46100       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    1900        input_2[0][0]                    
______________________________________________________________________________________________

In [13]:
att_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 100)    46100       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    1900        input_2[0][0]                    
____________________________________________________________________________________________

In [14]:
opti=Adam()
model.compile(loss=["categorical_crossentropy", "categorical_crossentropy"], optimizer=opti)

In [15]:
durations_network_input[100]

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 8, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [16]:
model.fit([notes_network_input, durations_network_input], [notes_network_output, durations_network_output],\
          epochs=10, batch_size=32, shuffle=True, validation_split = 0.2)
model.save("/content/drive/MyDrive/MODELS/keras_lstm&att.h5")

Epoch 1/10
692/692 [==============================] - 26s 32ms/step - loss: 4.5773 - dense_1_loss: 3.7202 - dense_2_loss: 0.8571 - val_loss: 3.9063 - val_dense_1_loss: 3.2655 - val_dense_2_loss: 0.6408
Epoch 2/10
692/692 [==============================] - 20s 29ms/step - loss: 4.0769 - dense_1_loss: 3.3931 - dense_2_loss: 0.6838 - val_loss: 3.7609 - val_dense_1_loss: 3.1574 - val_dense_2_loss: 0.6035
Epoch 3/10
692/692 [==============================] - 20s 29ms/step - loss: 3.8802 - dense_1_loss: 3.2496 - dense_2_loss: 0.6306 - val_loss: 3.6596 - val_dense_1_loss: 3.0963 - val_dense_2_loss: 0.5633
Epoch 4/10
692/692 [==============================] - 20s 29ms/step - loss: 3.7028 - dense_1_loss: 3.1051 - dense_2_loss: 0.5977 - val_loss: 3.7042 - val_dense_1_loss: 3.1517 - val_dense_2_loss: 0.5525
Epoch 5/10
692/692 [==============================] - 20s 28ms/step - loss: 3.5429 - dense_1_loss: 2.9778 - dense_2_loss: 0.5651 - val_loss: 3.6281 - val_dense_1_loss: 3.0739 - val_dense_2_los

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [17]:
model.save("/content/drive/MyDrive/MODELS/keras_lstm&att.h5")

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [18]:
def sample_with_temp(preds, temperature):

    if temperature == 0:
        return np.argmax(preds)
    else:
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        return np.random.choice(len(preds), p=preds)

In [19]:
how_to_start=["D3", 0.75]
temperature=0.2

notes_input=["START"]*(seq_len-1)+[how_to_start[0]]
durations_input=[0]*(seq_len-1)+[how_to_start[1]]
new_notes=[note2num_dic[how_to_start[0]]]
new_durations=[duration2num_dic[how_to_start[1]]]

notes_array=np.array([[note2num_dic[note_i] for note_i in notes_input]])
durations_array=np.array([[duration2num_dic[duration_i] for duration_i in durations_input]])

song_len=1
new_note=None
while new_note!=note2num_dic["END"] and song_len<200:
    note_pred, duration_pred=model.predict([notes_array, durations_array])
    new_note=sample_with_temp(note_pred[0], temperature)
    new_duration=sample_with_temp(duration_pred[0], temperature)
    notes_array=np.expand_dims(np.append(notes_array[0, 1:], new_note), 0)
    durations_array=np.expand_dims(np.append(durations_array[0, 1:], new_duration), 0)

    new_notes.append(new_note)
    new_durations.append(new_duration)
    song_len+=1

In [20]:
note_pred.shape

(1, 461)

In [21]:
len(new_notes)

200

In [22]:
new_notes

[206,
 356,
 261,
 206,
 206,
 284,
 445,
 445,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 356,
 356,
 356,
 356,
 388,
 388,
 445,
 72,
 72,
 445,
 445,
 445,
 356,
 445,
 445,
 445,
 72,
 72,
 72,
 445,
 356,
 445,
 388,
 356,
 356,
 261,
 356,
 356,
 261,
 261,
 356,
 206,
 261,
 206,
 206,
 54,
 161,
 161,
 161,
 161,
 54,
 368,
 54,
 206,
 54,
 54,
 368,
 161,
 368,
 368,
 356,
 356,
 356,
 356,
 161,
 401,
 340,
 356,
 445,
 445,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 445,
 445,
 445,
 445,
 445,
 72,
 388,
 445,
 388,
 445,
 72,
 388,
 445,
 445,
 388,
 356,
 388,
 388,
 356,
 356,
 388,
 356,
 356,
 356,
 356,
 356,
 261,
 206,
 161,
 161,
 161,
 161,
 161,
 161,
 161,
 368,
 368,
 368,
 368,
 401,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 401,
 368,
 368,
 401,
 368,
 340,
 368,
 401,
 368,
 78,
 261,
 356,
 445,
 356,
 356,
 356,
 356,
 261,
 261,
 388,
 388,
 356,
 

In [23]:
notes_output=[num2note_dic[note_i] for note_i in new_notes]
durations_output=[num2duration_dic[duration_i] for duration_i in new_durations]

In [24]:
import time
import os

In [25]:
midi_stream = stream.Stream()

# create note and chord objects based on the values generated by the model
for note_i, duration_i in zip(notes_output, durations_output):

    # pattern is a chord
    if ('.' in note_i):
        notes_in_chord = note_i.split('.')
        chord_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(current_note)
            new_note.duration = duration.Duration(duration_i)
            new_note.storedInstrument = instrument.Violoncello()
            chord_notes.append(new_note)
        new_chord = chord.Chord(chord_notes)
        midi_stream.append(new_chord)
    elif note_i == 'rest':
    # pattern is a rest
        new_note = note.Rest()
        new_note.duration = duration.Duration(duration_i)
        new_note.storedInstrument = instrument.Violoncello()
        midi_stream.append(new_note)
    elif note_i != 'START':
    # pattern is a note
        new_note = note.Note(note_i)
        new_note.duration = duration.Duration(duration_i)
        new_note.storedInstrument = instrument.Violoncello()
        midi_stream.append(new_note)



midi_stream = midi_stream.chordify()
timestr = time.strftime("%Y%m%d-%H%M%S")
output_folder="/content/drive/MyDrive/MIDI/"
midi_stream.write('midi', fp=os.path.join(output_folder, 'output-' + timestr + '.mid'))

'/content/drive/MyDrive/MIDI/output-20211018-141402.mid'

In [26]:
# how_to_start=["D3", 0.75]
# notes_input=["START"]*(seq_len-1)+[how_to_start[0]]
# durations_input=[0]*(seq_len-1)+[how_to_start[1]]
# new_notes=[note2num_dic[how_to_start[0]]]
# new_durations=[duration2num_dic[how_to_start[1]]]

# notes_array=np.array([[note2num_dic[note_i] for note_i in notes_input]])
# durations_array=np.array([[duration2num_dic[duration_i] for duration_i in durations_input]])
# new_note, new_duration=model.predict([notes_array, durations_array])
# new_note=np.argmax(new_note)
# new_duration=np.argmax(new_duration)